In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

In [ ]:
# Создание объекта UserAgent
user_agent = UserAgent()

# Генерация случайного user-agent
random_ua = user_agent.random

# Создание объекта ChromeOptions
options = webdriver.ChromeOptions()

# Установка случайного user-agent в объект ChromeOptions
options.add_argument(f'user-agent={random_ua}')

# Отключение функции AutomationControlled для браузера Chrome
options.add_argument('--disable-blink-features=AutomationControlled')

# Путь до ChromeDriver
s = Service(executable_path=r"C:\Users\asv37\Desktop\chromedriver.exe")

# Создание объекта webdriver.Chrome с заданными опциями
driver = webdriver.Chrome(service=s, options=options)

In [ ]:
# Создаем пустой список
data = []

# Цикл перелистывает страницы сайта
for p in range(1, 3382):
    print(p)
    
    # Загрузка страницы сайта
    driver.get(f"https://kinogo.biz/filmy/page/{p}/")
    
    # Время паузы в перелистывании, для того чтобы не перегрузить сервер
    time.sleep(2)
    
    # Формат кода
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Пкредаем переменной фрагмент кода с необходимой информацией по соответствующему тегу и классу
    films = soup.find_all('div', {'class': 'shortstory'})
    
    # Цикл для обработки каждого фильма
    for film in films:
        
        # Цикл для обработки каждого фильма, в случае отсутствия ставит значение None
        try:
            title = film.find('div', {'class': 'shortstory__title'}).find('h2').text.split('(')[0]
        except:
            title = None
            
        # Получение всех объектов с тегом span
        info_spans = film.find('div', {'class': 'shortstory__info-wrapper'}).find_all('span')
        
        # Получение информации о фильме в виде словаря
        # Создание пустого словаря
        info_dict = {}
        
        # Цикл for проходит по каждому элементу списка info_spans.
        for b in info_spans:
        
            # В блоке try находится поиск элемента <b> в текущем элементе списка info_spans и сохранение его текста в переменную key 
            # Далее, в переменную value записывается текст текущего элемента списка info_spans, 
            # из которого удаляется строка с ключом key и лишние пробелы с помощью методов replace и strip 
            # Наконец, создается ключ-значение в словаре info_dict с ключом key и значением value
            # В блоке except происходит переход к следующей итерации цикла, если не было найдено элемента <b>
            # в текущем элементе списка info_spans или возникла другая ошибка
            # Исключения в этом блоке позволяют избежать ошибок, если ключи и значения не были найдены.
            try:
                key = b.find('b').text
                value = b.text.replace(key, '').strip()
                info_dict[key] = value
            except:
                continue
            
            # Получает значение, связанное с ключом, 'Год выпуска:' из info_dict словаря и присваивает его переменной year
            # Если ключ не найден, year присваивается None
            try:
                year = info_dict.get('Год выпуска:')
            except:
                year = None
            try:
                country = info_dict.get('Страна:')
            except:
                country = None
            try:
                genre = info_dict.get('Жанр:')
            except:
                genre = None
            try:
                duration = info_dict.get('Продолжительность:')
            except:
                duration = None
            try:
                quality = info_dict.get('Качество:')
            except:
                quality = None
                
        # Переменной присвоивается текст, хранящийся в указанном классе
        try:
            watch = film.find('div', {'class':'total_meta-views'}).text
        except:
            watch = None
        try:
            score = film.find('span', {'class':'ignore-select'}).text
        except:
            score = None
        try:    
            rating_kp = film.find('span', {'class': 'kp'}).text.split()[1]
        except:
            rating_kp = None
        try:
            rating_imdb = film.find('span', {'class': 'imdb'}).text.split()[1]
        except:
            rating_imdb = None
        try:    
            rating_kinogo = film.find('span', {'class': 'rating__results'}).text.split()[0].split('/')[0]
        except:
            rating_kinogo = None
            
        # Cписок заполняется полученным данными в переменных
        data.append([title, year, country, genre, duration, quality, watch, score, rating_kp, rating_imdb, rating_kinogo])
        print(len(data))

In [58]:
# Создаем названия колонок
header = ['title', 'year', 'country', 'genre', 
          'duration', 'quality', 'watch', 'score', 
          'rating_kp', 'rating_imdb', 'rating_kinogo']

# Создаем датафрейм из собранных данных
df = pd.DataFrame(data, columns=header)

# Сохраняем датафрейм в формате CSV с разделением ';' и кодировкой 'utf8'
df.to_csv(r'kino.csv', sep=';', encoding='utf8')

In [60]:
dt = pd.read_csv('kino.csv', sep=';')
dt.head()

,Unnamed: 0,title,year,country,genre,duration,quality,watch,score,rating_kp,rating_imdb,rating_kinogo
0,0,"Анжелика, маркиза ангелов",2013,Франция,Приключения / Мелодрама / Фильмы,113 мин. / 01:53,BluRay,29 227,49,6.2,6.1,4.3
1,1,В центре внимания,2015,США,Криминал / Драма / Биография / Фильмы,129 мин. / 02:09,BDRip 1080p,29 181,58,7.6,8.1,4.5
2,2,Торчки,2001,США,Комедия / Фэнтези / Фильмы,93 мин. / 01:33,BDRip 1080p,29 448,41,6.8,6.2,3.6
3,3,Земля мечтаний,2022,Италия,Новинки / Фильмы / Мюзикл,NaN,WEB-DL,318,2,NaN,NaN,4.5
4,4,Полуночное кафе: Исчезновение невесты,2022,Корея Южная,Новинки / Фильмы / Мелодрама / Фэнтези,103 мин.,WEB-DL,197,2,NaN,NaN,3.5
